In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion, iterate_structure
from microphone import record_audio
from typing import Tuple, Callable, List
from numba import njit
import librosa

%matplotlib notebook

In [2]:
import ipynb
from ipynb.fs.full.audio_to_samples import mp3_path_to_samples, microphone_audio_to_samples
from ipynb.fs.full.spectro_peaks import _peaks, local_peak_locations, local_peaks_mask
from ipynb.fs.full.Fingerprints_from_Peaks import create_fingerprints

In [62]:
##### skip
from collections import Counter
import pickle
def load_or_save_song_database(*, action: str, file_name: str, song_database=None):
    if action == "load":
        song_database_file = open(file_name, "rb")
        song_database = pickle.load(song_database_file)
        song_database_file.close()
        return song_database
    elif action == "save":
        song_database_file = open(file_name, "wb")
        pickle.dump(song_database, song_database_file)
        song_database_file.close()

def add_fingerprint_song_pair(
    fingerprint: tuple, song_name: str, time_stamp: float, song_database: dict
):
    if fingerprint in song_database.keys():
        song_database[fingerprint].append((song_name, time_stamp))
    else:
        song_database[fingerprint] = [(song_name, time_stamp)]

In [63]:
import pickle 
import ipynb

from ipynb.fs.full.audio_to_samples import mp3_path_to_samples, microphone_audio_to_samples
from ipynb.fs.full.spectro_peaks import _peaks, local_peak_locations, local_peaks_mask
from ipynb.fs.full.Fingerprints_from_Peaks import create_fingerprints
#/Users/koalum/Documents/GitHub/Week1Capstone/songs/allstar.mp3

print("~ Welcome to COGZAM ~")

var = input("Would you like to (1)record or (2)upload: ")
if var == str(1):
    samples = microphone_audio_to_samples(15)
elif var == str(2):
    file_path = input("Input your file path: ")
    samples = mp3_path_to_samples(file_path)
else:
    print("That's not an option, please type 1 or 2.")

~ Welcome to COGZAM ~
Would you like to (1)record or (2)upload: 1
Using input device 'AirPods'
Recording ended


In [64]:
sampling_rate = 44100
recorded_audio = samples

S, freqs, times = mlab.specgram(
    recorded_audio,
    NFFT=4096,
    Fs=sampling_rate,
    window=mlab.window_hanning,
    noverlap=4096 // 2,
)

peaks = local_peaks_mask(S, cutoff = np.percentile(S, 75))
song_fingerprints = create_fingerprints(peaks, 15)
song_database = load_or_save_song_database(action = "load", file_name = "songs_db.pkl")

In [66]:
def get_song(fingerprints: list, song_database: dict) -> str:
    potential_songs = []
    
    for fingerprint in fingerprints:
        if fingerprint[0] in song_database:
            for song_name, time_stamp in song_database[fingerprint[0]]:
                potential_songs.append(song_name)
    
    song_matches = Counter(potential_songs)
    
    print(fingerprint)
    print(len(fingerprints))
    print(song_matches)
    if (song_matches.most_common(1)[0][1] - song_matches.most_common(2)[1][1] > 20) and song_matches.most_common(1)[0][1] > 50:
        return song_matches.most_common(1)[0]
    else:
        return "NO SONG MATCH"

In [67]:
output = get_song(song_fingerprints, song_database)
if output == "NO SONG MATCH":
    decision = input("No matched song, would you like to add it to the database? Yes or no: ")
    if decision.lower() == "yes":
        name = input("Enter song name: ")
        load_or_save_song_database(action = "save", file_name = name, song_database = song_database)
    elif decision.lower() == "no":
        print("Thank you for using COGZAM")
else:
    print("Your song matched: " + str(output[0])) # fix this later

((633, 40, 4), 316)
2505
Counter({'allstar.mp3': 102, 'Undertale OST - Snowy.mp3': 3, 'magicschoolbus.mp3': 2})
Your song matched: allstar.mp3
